# 🎯 EXTRACT WAVE 1 RESULTS - START ML NOW!

## Purpose:
Extract and consolidate Wave 1 results (12 batches × 35K = 420K images) into a ready-to-use CSV for immediate ML pipeline and analysis.

## Wave 1 Batches:
- Shards 0000-0011 (12 batches)
- 35,000 images per batch
- Total: ~420,000 images
- All completed successfully ✅

## Output:
- **COMICS_OCR_WAVE1_sorted.csv** - Ready for ML training
- Sorted by comic/page/panel
- Full OCR data with bubbles and text

## Run Time: ~5-7 minutes

---

In [1]:
# Import libraries
from pathlib import Path
from google.cloud import storage
import json
import csv
import re
from tqdm import tqdm
import pandas as pd

print("✅ All libraries imported")

✅ All libraries imported


In [2]:
# Configuration
PROJECT_ID = "fluent-justice-478703-f8"
LOCATION = "us-central1"
BUCKET = "harshasekar-comics-data"

# Wave 1 specific paths
BATCH_OUTPUT_PREFIX = "ocr_outputs/optimized_35k"

# Wave 1: Shards 0000-0011 (12 batches)
WAVE1_SHARDS = list(range(0, 12))  # 0, 1, 2, ..., 11

# Output files
OUTPUT_DIR = Path(".")
WAVE1_CSV = OUTPUT_DIR / "COMICS_OCR_WAVE1_sorted.csv"
WAVE1_STATS_CSV = OUTPUT_DIR / "COMICS_OCR_WAVE1_stats.csv"

print("="*80)
print("WAVE 1 EXTRACTION CONFIGURATION")
print("="*80)
print(f"Project: {PROJECT_ID}")
print(f"Bucket: gs://{BUCKET}/")
print(f"Output prefix: {BATCH_OUTPUT_PREFIX}")
print(f"Wave 1 shards: {WAVE1_SHARDS[0]}-{WAVE1_SHARDS[-1]} ({len(WAVE1_SHARDS)} batches)")
print(f"Expected images: ~{len(WAVE1_SHARDS) * 35000:,}")
print(f"Output CSV: {WAVE1_CSV}")
print("="*80)

WAVE 1 EXTRACTION CONFIGURATION
Project: fluent-justice-478703-f8
Bucket: gs://harshasekar-comics-data/
Output prefix: ocr_outputs/optimized_35k
Wave 1 shards: 0-11 (12 batches)
Expected images: ~420,000
Output CSV: COMICS_OCR_WAVE1_sorted.csv


In [3]:
# Connect to GCS
print("📡 Connecting to Google Cloud Storage...")
client = storage.Client(project=PROJECT_ID)
bucket = client.bucket(BUCKET)
print("✅ Connected successfully!")

📡 Connecting to Google Cloud Storage...
✅ Connected successfully!


In [15]:
def find_prediction_file(shard_num: int) -> str:
    """
    Finds the predictions.jsonl file for a specific shard number.
    Searches in: ocr_outputs/optimized_35k/wave1_job_XXXX/
    """
    # Try wave1 path first
    prefix = f"{BATCH_OUTPUT_PREFIX}/wave1_job_{shard_num:04d}/"
    blobs = list(bucket.list_blobs(prefix=prefix))
    
    for blob in blobs:
        if "prediction" in blob.name.lower() and blob.name.endswith(".jsonl"):
            return blob.name
    
    # Try alternative naming patterns
    prefix = f"{BATCH_OUTPUT_PREFIX}/job_{shard_num:04d}/"
    blobs = list(bucket.list_blobs(prefix=prefix))
    
    for blob in blobs:
        if "prediction" in blob.name.lower() and blob.name.endswith(".jsonl"):
            return blob.name
    
    return None


def parse_gemini_ocr_line(jsonl_line: str) -> dict:
    """
    Parses a single JSONL line from Gemini batch output.
    Produces clean, readable text preserving ALL original punctuation.
    """
    try:
        rec = json.loads(jsonl_line)
    except json.JSONDecodeError:
        return None
    
    # Extract custom_id
    custom_id = rec.get("custom_id", "")
    parts = custom_id.split("-")
    
    if len(parts) < 3:
        return None
    
    comic_no = parts[0]
    page_no = parts[1]
    panel_no = parts[2]
    
    img_path = f"raw_panel_images/{comic_no}/{page_no}_{panel_no}.jpg"
    
    # Extract response
    response = rec.get("response", {})
    candidates = response.get("candidates", [])
    
    if not candidates:
        return {
            "comic_no": comic_no,
            "page_no": page_no,
            "panel_no": panel_no,
            "img_path": img_path,
            "agg_text": "",
            "bubble_count": 0,
            "bubbles_json": json.dumps({"bubbles": []}),
        }
    
    content = candidates[0].get("content", {})
    parts_list = content.get("parts", [])
    
    if not parts_list:
        return {
            "comic_no": comic_no,
            "page_no": page_no,
            "panel_no": panel_no,
            "img_path": img_path,
            "agg_text": "",
            "bubble_count": 0,
            "bubbles_json": json.dumps({"bubbles": []}),
        }
    
    raw_text = parts_list[0].get("text", "")
    
    # Parse JSON response
    bubbles_data = {"bubbles": []}
    try:
        match = re.search(r"```(?:json)?\s*({.*?})\s*```", raw_text, re.DOTALL)
        if match:
            bubbles_data = json.loads(match.group(1))
        else:
            bubbles_data = json.loads(raw_text)
    except (json.JSONDecodeError, AttributeError):
        pass
    
    bubbles = bubbles_data.get("bubbles", [])
    
    # CLEAN TEXT PROCESSING - Preserves ALL punctuation
    cleaned_texts = []
    for bubble in bubbles:
        text = bubble.get("text", "")
        
        # Handle None
        if text is None:
            continue
        
        # Convert to string
        text = str(text).strip()
        
        if not text:
            continue
        
        # Remove literal \n escape sequences and replace with spaces
        text = text.replace("\\n", " ")
        
        # Replace actual newlines with spaces
        text = text.replace("\n", " ")
        
        # Replace multiple spaces with single space
        text = " ".join(text.split())
        
        # Remove only tab and carriage return escapes
        text = text.replace("\\t", " ")
        text = text.replace("\\r", " ")
        
        cleaned_texts.append(text)
    
    # Join all bubble text with space separator
    # Preserve original punctuation - no modifications
    agg_text = " ".join(cleaned_texts)
    
    return {
        "comic_no": comic_no,
        "page_no": page_no,
        "panel_no": panel_no,
        "img_path": img_path,
        "agg_text": agg_text,
        "bubble_count": len(bubbles),
        "bubbles_json": json.dumps(bubbles_data),
    }

In [16]:
# Find prediction files for Wave 1
print("🔍 Finding prediction files for Wave 1 batches...\n")

wave1_files = {}
missing_shards = []

for shard_num in WAVE1_SHARDS:
    pred_file = find_prediction_file(shard_num)
    
    if pred_file:
        wave1_files[shard_num] = pred_file
        print(f"✅ Shard {shard_num:04d}: {pred_file}")
    else:
        missing_shards.append(shard_num)
        print(f"⚠️  Shard {shard_num:04d}: No prediction file found")

print(f"\n{'='*80}")
print(f"Found: {len(wave1_files)}/{len(WAVE1_SHARDS)} prediction files")
if missing_shards:
    print(f"⚠️  Missing shards: {missing_shards}")
    print(f"   Check if these batches completed successfully")
else:
    print("✅ All Wave 1 prediction files found!")
print(f"{'='*80}")

🔍 Finding prediction files for Wave 1 batches...

✅ Shard 0000: ocr_outputs/optimized_35k/job_0000/prediction-model-2025-11-21T11:50:37.275707Z/predictions.jsonl
✅ Shard 0001: ocr_outputs/optimized_35k/job_0001/prediction-model-2025-11-21T11:50:37.387656Z/predictions.jsonl
✅ Shard 0002: ocr_outputs/optimized_35k/job_0002/prediction-model-2025-11-21T11:50:37.494714Z/predictions.jsonl
✅ Shard 0003: ocr_outputs/optimized_35k/job_0003/prediction-model-2025-11-21T11:50:37.614252Z/predictions.jsonl
✅ Shard 0004: ocr_outputs/optimized_35k/job_0004/prediction-model-2025-11-21T11:50:37.725457Z/predictions.jsonl
✅ Shard 0005: ocr_outputs/optimized_35k/job_0005/prediction-model-2025-11-21T11:50:37.825309Z/predictions.jsonl
✅ Shard 0006: ocr_outputs/optimized_35k/job_0006/prediction-model-2025-11-21T11:50:37.941752Z/predictions.jsonl
✅ Shard 0007: ocr_outputs/optimized_35k/job_0007/prediction-model-2025-11-21T11:50:38.070233Z/predictions.jsonl
✅ Shard 0008: ocr_outputs/optimized_35k/job_0008/predi

In [17]:
# Extract OCR results from Wave 1
print("\n📥 Extracting OCR results from Wave 1 batches...\n")

# Open CSV writer
csv_file = WAVE1_CSV.open("w", newline="", encoding="utf-8")
writer = csv.writer(csv_file)
writer.writerow([
    "comic_no", "page_no", "panel_no",
    "img_path", "agg_text", "bubble_count", "bubbles_json"
])

# Track stats
batch_stats = []
total_panels = 0
total_tokens = 0

# Process each shard in order
for shard_num in tqdm(sorted(wave1_files.keys()), desc="Processing Wave 1"):
    blob_name = wave1_files[shard_num]
    
    try:
        # Download prediction file
        blob = bucket.blob(blob_name)
        if not blob.exists():
            print(f"\n⚠️  Blob not found: {blob_name}")
            batch_stats.append({
                "shard_num": shard_num,
                "num_panels": 0,
                "total_tokens": 0,
                "status": "blob_not_found"
            })
            continue
        
        content = blob.download_as_text(encoding="utf-8")
        lines = content.strip().split("\n")
        
        # Parse each line
        batch_panels = 0
        batch_tokens = 0
        
        for line in lines:
            if not line.strip():
                continue
            
            # Extract OCR data
            parsed = parse_gemini_ocr_line(line)
            if not parsed:
                continue
            
            # Write to CSV
            writer.writerow([
                parsed["comic_no"],
                parsed["page_no"],
                parsed["panel_no"],
                parsed["img_path"],
                parsed["agg_text"],
                parsed["bubble_count"],
                parsed["bubbles_json"],
            ])
            
            batch_panels += 1
            total_panels += 1
            
            # Track tokens
            try:
                rec = json.loads(line)
                um = rec.get("response", {}).get("usageMetadata", {})
                batch_tokens += um.get("totalTokenCount", 0)
            except:
                pass
        
        total_tokens += batch_tokens
        
        batch_stats.append({
            "shard_num": shard_num,
            "num_panels": batch_panels,
            "total_tokens": batch_tokens,
            "status": "success"
        })
    
    except Exception as e:
        print(f"\n❌ Error processing shard {shard_num}: {e}")
        batch_stats.append({
            "shard_num": shard_num,
            "num_panels": 0,
            "total_tokens": 0,
            "status": f"error: {str(e)}"
        })

csv_file.close()

print(f"\n✅ Extracted {total_panels:,} panels from Wave 1")


📥 Extracting OCR results from Wave 1 batches...



Processing Wave 1: 100%|██████████| 12/12 [00:58<00:00,  4.86s/it]



✅ Extracted 420,000 panels from Wave 1


In [18]:
# Save batch statistics
with WAVE1_STATS_CSV.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["shard_num", "num_panels", "total_tokens", "status"])
    for stat in batch_stats:
        writer.writerow([
            stat["shard_num"],
            stat["num_panels"],
            stat["total_tokens"],
            stat["status"]
        ])

print(f"✅ Stats saved to: {WAVE1_STATS_CSV}")

✅ Stats saved to: COMICS_OCR_WAVE1_stats.csv


In [19]:
# Sort CSV by comic/page/panel for ML pipeline
print("📊 Sorting CSV by comic_no, page_no, panel_no...")

df = pd.read_csv(WAVE1_CSV)

# Convert to numeric for proper sorting
df["comic_no"] = pd.to_numeric(df["comic_no"], errors="coerce")
df["page_no"] = pd.to_numeric(df["page_no"], errors="coerce")
df["panel_no"] = pd.to_numeric(df["panel_no"], errors="coerce")

# Sort
df = df.sort_values(["comic_no", "page_no", "panel_no"], ascending=[True, True, True])

# Save sorted version
df.to_csv(WAVE1_CSV, index=False)

print(f"✅ Sorted CSV saved: {WAVE1_CSV}")
print(f"   Total rows: {len(df):,}")

📊 Sorting CSV by comic_no, page_no, panel_no...
✅ Sorted CSV saved: COMICS_OCR_WAVE1_sorted.csv
   Total rows: 420,000


In [13]:
# Print final summary
print("\n" + "="*80)
print("🎉 WAVE 1 EXTRACTION COMPLETE!")
print("="*80)
print()
print(f"Total panels extracted:     {total_panels:,}")
print(f"Total tokens used:          {total_tokens:,}")
print(f"Average tokens per panel:   {total_tokens/max(total_panels, 1):.1f}")
print()
print("OUTPUT FILES:")
print(f"  📄 Main CSV:              {WAVE1_CSV}")
print(f"  📊 Stats:                 {WAVE1_STATS_CSV}")
print()
print("BATCH BREAKDOWN:")
for stat in batch_stats:
    status_icon = "✅" if stat["status"] == "success" else "⚠️"
    avg_tokens = stat["total_tokens"] / max(stat["num_panels"], 1)
    print(f"  {status_icon} Shard {stat['shard_num']:04d}: {stat['num_panels']:6,} panels | "
          f"{stat['total_tokens']:10,} tokens | {avg_tokens:6.1f} tok/panel | {stat['status']}")
print()
print("="*80)
print()
print("✅ CSV is ready for ML pipeline!")
print("📈 Start training with this dataset immediately")
print("🔄 Wave 2 and 3 can be merged later when they complete")
print()
print(f"💾 Load with: df = pd.read_csv('{WAVE1_CSV}')")
print("="*80)


🎉 WAVE 1 EXTRACTION COMPLETE!

Total panels extracted:     420,000
Total tokens used:          717,192,667
Average tokens per panel:   1707.6

OUTPUT FILES:
  📄 Main CSV:              COMICS_OCR_WAVE1_sorted.csv
  📊 Stats:                 COMICS_OCR_WAVE1_stats.csv

BATCH BREAKDOWN:
  ✅ Shard 0000: 35,000 panels | 63,056,749 tokens | 1801.6 tok/panel | success
  ✅ Shard 0001: 35,000 panels | 59,600,134 tokens | 1702.9 tok/panel | success
  ✅ Shard 0002: 35,000 panels | 59,649,446 tokens | 1704.3 tok/panel | success
  ✅ Shard 0003: 35,000 panels | 58,917,068 tokens | 1683.3 tok/panel | success
  ✅ Shard 0004: 35,000 panels | 58,734,953 tokens | 1678.1 tok/panel | success
  ✅ Shard 0005: 35,000 panels | 60,559,569 tokens | 1730.3 tok/panel | success
  ✅ Shard 0006: 35,000 panels | 54,521,021 tokens | 1557.7 tok/panel | success
  ✅ Shard 0007: 35,000 panels | 62,864,621 tokens | 1796.1 tok/panel | success
  ✅ Shard 0008: 35,000 panels | 62,015,756 tokens | 1771.9 tok/panel | success
  ✅ 

In [20]:
# Preview the data
print("\n📊 Data Preview:\n")
df.head(10)


📊 Data Preview:



,comic_no,page_no,panel_no,img_path,agg_text,bubble_count,bubbles_json
24764,0,0,0,raw_panel_images/0/0_0.jpg,JULY No.30 10c BINKS ARMORED DELIVERY MAIN ST.,3,"{""bubbles"": [{""text"": ""JULY\nNo.30\n10c"", ""raw..."
1901,0,0,1,raw_panel_images/0/0_1.jpg,G ACCOUNT OF ss Wiggins MAN WHO OKE LAW OF AVI...,6,"{""bubbles"": [{""text"": ""G ACCOUNT OF\nss Wiggin..."
15839,0,2,0,raw_panel_images/0/2_0.jpg,"CK HIM ,PLAS! CK HIM WN! PLASTIC MAN",2,"{""bubbles"": [{""text"": ""CK HIM\n,PLAS!\nCK HIM\..."
24564,0,2,1,raw_panel_images/0/2_1.jpg,"PLASTICMAN IN HIS CAREER AS AS CRIME-FIGHTER, ...",3,"{""bubbles"": [{""text"": ""PLASTICMAN"", ""raw_text""..."
14543,0,3,0,raw_panel_images/0/3_0.jpg,NOW IF I PUT THIS HERE... AND THIS ONE HERE...,1,"{""bubbles"": [{""text"": ""NOW IF I PUT THIS HERE...."
33637,0,3,1,raw_panel_images/0/3_1.jpg,AND ADJUST THIS SPRING...,1,"{""bubbles"": [{""text"": ""AND ADJUST THIS SPRING...."
15840,0,3,2,raw_panel_images/0/3_2.jpg,THAT DOES IT! IT'S FINISHED AND READY FOR THE ...,1,"{""bubbles"": [{""text"": ""THAT DOES IT! IT'S FINI..."
4158,0,3,3,raw_panel_images/0/3_3.jpg,I'VE BEEN WORKING ON IT FOR DAYS! I MUST GET A...,1,"{""bubbles"": [{""text"": ""I'VE BEEN WORKING ON IT..."
10541,0,3,4,raw_panel_images/0/3_4.jpg,"AH, I FEEL BETTER ALREADY! NOW FOR A BRISK WAL...",1,"{""bubbles"": [{""text"": ""AH, I FEEL BETTER\nALRE..."
23050,0,3,5,raw_panel_images/0/3_5.jpg,"And WEIGHTLESS WIGGINS, THUG AND PROFESSIONAL ...",2,"{""bubbles"": [{""text"": ""And WEIGHTLESS WIGGINS,..."


In [ ]:
# Basic statistics for ML pipeline
print("\n📈 Dataset Statistics for ML:\n")
print(f"Total samples: {len(df):,}")
print(f"Unique comics: {df['comic_no'].nunique():,}")
print(f"\nText Distribution:")
print(f"  Panels with text: {(df['bubble_count'] > 0).sum():,} ({(df['bubble_count'] > 0).sum()/len(df)*100:.1f}%)")
print(f"  Empty panels: {(df['bubble_count'] == 0).sum():,} ({(df['bubble_count'] == 0).sum()/len(df)*100:.1f}%)")
print(f"  Average bubbles per panel: {df['bubble_count'].mean():.2f}")
print(f"  Max bubbles in one panel: {df['bubble_count'].max()}")

# Text length analysis
df['text_length'] = df['agg_text'].str.len()
text_panels = df[df['text_length'] > 0]

print(f"\nText Length Statistics:")
print(f"  Mean: {text_panels['text_length'].mean():.1f} characters")
print(f"  Median: {text_panels['text_length'].median():.1f} characters")
print(f"  Max: {text_panels['text_length'].max()} characters")

# Sample size for ML
print(f"\n🤖 ML Training Recommendations:")
print(f"  Available samples: {len(df):,}")
print(f"  Suggested train/val split: 80/20")
print(f"    - Training set: {int(len(df) * 0.8):,} samples")
print(f"    - Validation set: {int(len(df) * 0.2):,} samples")

In [ ]:
# Comic distribution analysis
print("\n📚 Comic Distribution (Top 20):")
print()
comic_counts = df['comic_no'].value_counts().head(20)
for comic_id, count in comic_counts.items():
    print(f"  Comic {int(comic_id):4d}: {count:5,} panels")

---
# ✅ WAVE 1 READY FOR ML!

## What You Have:
- ✅ **~420,000 images** with complete OCR data (Wave 1)
- ✅ **Sorted by comic/page/panel** for sequential reading
- ✅ **Full bubble data** with text, types, and structured JSON
- ✅ **Ready for immediate ML training**

## Next Steps:

### 1. Start ML Pipeline NOW
```python
import pandas as pd

# Load Wave 1 data
df = pd.read_csv('COMICS_OCR_WAVE1_sorted.csv')

# Create train/val split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Start training!
```

### 2. Sample 100 Comics for Fine-tuning
```python
# Get 100 random comics with most panels
top_comics = df['comic_no'].value_counts().head(100).index.tolist()
sample_df = df[df['comic_no'].isin(top_comics)]
sample_df.to_csv('COMICS_OCR_SAMPLE_100.csv', index=False)
```

### 3. Continue with Wave 2 & 3 in Parallel
- Wave 2 and 3 results can be merged later
- Continue training with Wave 1 data
- Final model can be updated with full 1.2M dataset

## Dataset Info:
- **Format**: CSV with structured OCR data
- **Size**: ~420K samples (Wave 1)
- **Features**: comic_no, page_no, panel_no, img_path, agg_text, bubble_count, bubbles_json
- **Ready for**: LLaVA, OpenFlamingo, or custom comic OCR models

---

## 🎯 YOU CAN NOW START YOUR ML PIPELINE!

No need to wait for Wave 2 and 3 - **start training NOW with 420K images!**

---